<a href="https://colab.research.google.com/github/Nick088Official/Mistral-Google-Colab/blob/main/Mistral_Manual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run Mistral 7B AI Models

GPU REQUIRED FOR THIS GOOGLE COLAB

Google Colab Made by [Nick088](https://linktr.ee/Nick088)

In [ ]:
#@title Install & Load Dependencies, Model
!pip install einops
!pip install accelerate
!pip install huggingface_hub
import torch
from IPython.display import clear_output
from huggingface_hub import hf_hub_download

if torch.cuda.is_available():
    device = "cuda"
    print("Using GPU")
else:
    device = "cpu"
    print("Using CPU")

Mistral_Model = "Mistral-7B-Instruct-v0.2" #@param ['Mistral-7B-v0.1', 'Mistral-7B-Instruct-v0.2']

#@markdown The normal version are the official ones from Mistral and they work only with colabs paid gpus , the other ones are GGUF quantized works on colab free gpu, compressed to consume less ram.

GGUF_Format = True #@param {type:"boolean"}

GGUF_Model_Version = "Q2_K" #@param ['Q2_K', 'Q3_K_S', 'Q3_K_M', 'Q3_K_L', 'Q4_0', 'Q4_K_M', 'Q4_K_S', 'Q5_0', 'Q5_K_S', 'Q5_K_M', 'Q6_K', 'Q8_0']

if GGUF_Format == False:
  from transformers import AutoModelForCausalLM, AutoTokenizer
  if device == "cuda":
    model = AutoModelForCausalLM.from_pretrained(f"mistralai/{Mistral_Model}", torch_dtype="auto", trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(f"mistralai/{Mistral_Model}", trust_remote_code=True)
  else:
    model = AutoModelForCausalLM.from_pretrained(f"mistralai/{Mistral_Model}", torch_dtype=torch.float32, device_map="cpu", trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(f"mistralai/{Mistral_Model}", trust_remote_code=True)
else:
  Mistral_Model = Mistral_Model.lower()
  !nvidia-smi
  !CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
  from huggingface_hub import hf_hub_download
  from llama_cpp import Llama
  model_name = f"TheBloke/{Mistral_Model}-GGUF"
  model_file = f"{Mistral_Model}.{GGUF_Model_Version}.gguf"
  model_path = hf_hub_download(model_name,
                             filename=model_file,
                             local_dir='/content')
  from llama_cpp import Llama
  llm = Llama(model_path=model_path,
            n_gpu_layers=-1)

clear_output()
if GGUF_Format == False:
  print(f"Downloaded {Mistral_Model}")
else:
  print(f"Downloaded {Mistral_Model} {GGUF_Model_Version}")

In [ ]:
#@title Run Inference
#@markdown Your Prompt
user_prompt = "What's Mistral?" #@param {type:"string"}

# Additional inputs
#@markdown Add your system prompt (prompt to personalize the AI) here or leave it empty if you wanna use the AI normally
system_prompt = "You are ShortAI, write short but concise responses"  #@param {type:"string"}
#@markdown The maximum number of tokens that the model will generate in response to your input
max_new_tokens = 2000 #@param {type:"slider", min:256, max:8192, step:1}
#@markdown Penalize repeated tokens, so make the AI repeat less of itself
repetition_penalty = 1.5 #@param {type:"slider", min:0.0, max:2, step:0.1}

# Concatenate system and user prompts
full_prompt = system_prompt + user_prompt

if GGUF_Format == False:
  # Tokenize the combined prompt
  inputs = tokenizer(full_prompt, return_tensors="pt", return_attention_mask=False)

  # Generate text based on the combined prompt and additional inputs
  outputs = model.generate(
      **inputs,
      max_new_tokens=max_new_tokens,
      repetition_penalty=repetition_penalty
      )

  text = tokenizer.batch_decode(outputs)[0]

  # Extract the generated text from the model output
  text = text[len(system_prompt):].strip()  # Remove the system prompt from the generated text
  clear_output()
  print(text)
else:
  response = llm(full_prompt, max_tokens=max_new_tokens, repeat_penalty=repetition_penalty)
  clear_output()
  print(response['choices'][0]['text'])